# EA-RoBERTa: Emotion-Aware RoBERTa with ESA + TF-IDF Gating + Multi-Labeling


In [1]:
import os
import re
import json
from typing import Optional, Dict, List
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, matthews_corrcoef, classification_report, f1_score
)
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler

from transformers import (
    RobertaTokenizerFast, RobertaModel,
    TrainingArguments, Trainer
)
from transformers.modeling_outputs import SequenceClassifierOutput

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Using device: cuda
GPU: NVIDIA A100-SXM4-80GB
Memory: 85.2 GB


## 1. Configuration

In [2]:
# =============================================================================
# MODEL CONFIGURATION (from paper)
# =============================================================================
CONFIG_GOEMOTIONS = {
    'model_name': 'roberta-base',
    'max_length': 256,
    'batch_size': 32,
    'learning_rate': 2e-5,
    'num_epochs': 10,
    'dropout': 0.3,
    'weight_decay': 0.01,
    'warmup_ratio': 0.1,
    'seed': 42,
    # TF-IDF settings
    'tfidf_threshold': 2.5,
    'tfidf_min_tokens': 4,
    # Gating
    'use_tfidf_gating': True,
    'multilabel_threshold': 0.5
}

# Create output directories
os.makedirs('models/ea_roberta_multiLabel', exist_ok=True)
os.makedirs('reports_multiLabel', exist_ok=True)

print("Configuration:")
for k, v in CONFIG_GOEMOTIONS.items():
    print(f"  {k}: {v}")

Configuration:
  model_name: roberta-base
  max_length: 256
  batch_size: 32
  learning_rate: 2e-05
  num_epochs: 10
  dropout: 0.3
  weight_decay: 0.01
  warmup_ratio: 0.1
  seed: 42
  tfidf_threshold: 2.5
  tfidf_min_tokens: 4
  use_tfidf_gating: True
  multilabel_threshold: 0.5


## 2. Text Preprocessing (Slang-Aware)

In [3]:
SLANG_MAP = {
    # Common abbreviations
    "u": "you", "ur": "your", "r": "are", "n": "and", "b": "be", "c": "see", "y": "why",
    "k": "okay", "ok": "okay", "pls": "please", "plz": "please", "thx": "thanks", "thnx": "thanks",
    "ty": "thank you", "bc": "because", "cuz": "because", "bcuz": "because", "bcoz": "because",
    "cos": "because", "coz": "because","w": "with", "wo": "without", "abt": "about", "bout": "about",
    "bf": "boyfriend", "gf": "girlfriend", "bff": "best friend", "rn": "right now",
    "imo": "in my opinion", "imho": "in my humble opinion", "tbh": "to be honest",
    "fyi": "for your information", "btw": "by the way", "afaik": "as far as i know",
    "irl": "in real life", "jk": "just kidding", "omw": "on my way", "brb": "be right back",
    "gtg": "got to go", "g2g": "got to go", "ttyl": "talk to you later", "nvm": "never mind",
    "idk": "i do not know", "idc": "i do not care", "idgaf": "i do not care", "dm": "direct message",
    "rt": "retweet", "fb": "facebook", "ig": "instagram", "yt": "youtube", "smh": "shaking my head",
    "fomo": "fear of missing out", "yolo": "you only live once", "bae": "babe", "fam": "family",
    "lit": "exciting", "slay": "amazing", "goat": "greatest of all time", "af": "very",
    "asf": "very", "lowkey": "somewhat", "highkey": "very", "srsly": "seriously", "tho": "though",
    "thru": "through", "kinda": "kind of", "sorta": "sort of", "gonna": "going to",
    "wanna": "want to", "gotta": "got to", "coulda": "could have", "shoulda": "should have",
    "woulda": "would have", "musta": "must have", "lemme": "let me", "gimme": "give me",
    "dunno": "do not know", "whatcha": "what are you", "gotcha": "got you", "outta": "out of", "lotta": "lot of",
    "lotsa": "lots of", "kinda": "kind of", "innit": "is it not", "aint": "is not", "ain't": "is not",
    "yall": "you all", "y'all": "you all",

    # Emotion-related expressions
    "lol": "laughing", "lmao": "laughing", "lmfao": "laughing", "rofl": "laughing",
    "roflmao": "laughing", "haha": "laughing", "hahaha": "laughing", "hehe": "laughing",
    "hihi": "laughing", "xd": "laughing", "xD": "laughing", "omg": "oh my god", "omfg": "oh my god",
    "wtf": "what the heck", "wth": "what the heck", "stfu": "shut up", "ugh": "frustrated",
    "meh": "indifferent", "oof": "ouch", "yikes": "shocked", "eww": "disgusted", "aww": "touched",
    "yay": "excited", "woohoo": "excited", "woah": "surprised", "whoa": "surprised",
    "wow": "surprised", "damn": "frustrated", "dammit": "frustrated", "damnit": "frustrated",

    # Missing apostrophe contractions
    "im": "i am", "ive": "i have", "id": "i would", "ill": "i will", "youre": "you are",
    "youve": "you have", "youd": "you would", "youll": "you will", "hes": "he is",
    "shes": "she is", "thats": "that is", "whats": "what is", "whos": "who is",
    "wheres": "where is", "heres": "here is", "theres": "there is", "theyre": "they are",
    "theyve": "they have", "theyd": "they would", "theyll": "they will", "weve": "we have",
    "wed": "we would", "dont": "do not", "doesnt": "does not", "didnt": "did not",
    "wont": "will not", "wouldnt": "would not", "couldnt": "could not", "shouldnt": "should not",
    "cant": "cannot", "cannot": "can not", "isnt": "is not", "arent": "are not",
    "wasnt": "was not", "werent": "were not", "hasnt": "has not", "havent": "have not",
    "hadnt": "had not", "mustnt": "must not", "lets": "let us",
}

EMOTICON_MAP = {
    ":)": " happy ", ":(": " sad ", ":D": " very happy ", ":-)": " happy ", ":-(": " sad ",
    ";)": " playful ", ";-)": " playful ", ":p": " playful ", ":P": " playful ", ":-p": " playful ",
    ":-P": " playful ", ":o": " surprised ", ":O": " surprised ", ":-o": " surprised ",
    ":-O": " surprised ", ":/": " unsure ", ":-/": " unsure ", ":\\": " unsure ", ":-\\": " unsure ",
    "<3": " love ", "</3": " heartbroken ", "xo": " love ", "xoxo": " love ",
}

NEGATION_WHITELIST = {"not", "no", "never", "n't", "none", "nothing", "nowhere"}
INTENSIFIERS = {"very", "so", "too", "really", "extremely", "quite"}

def normalize_elongated_words(text):
    """
    Reduce elongated characters to max 2 repetitions.
    'soooooo' -> 'soo', 'happyyy' -> 'happyy'
    This preserves some emphasis while normalizing.
    """

    return re.sub(r'(.)\1{2,}', r'\1\1', text)

def replace_slang(text):
    """
    Replace slang/abbreviations with full forms.
    Uses word boundaries to avoid partial replacements.
    """

    words = text.split()
    result = []
    for word in words:
        word_lower = word.lower()
        if word_lower in SLANG_MAP:
            result.append(SLANG_MAP[word_lower])
        else:
            result.append(word)
    return ' '.join(result)

def replace_emoticons(text):
    """Replace text emoticons with emotion words."""

    for emoticon, replacement in EMOTICON_MAP.items():
        text = text.replace(emoticon, replacement)
    return text

def clean_text(text):
    """Comprehensive text cleaning for social media data."""

    if not isinstance(text, str):
        return ""

    # 1. Replace emoticons with emotion words (before stripping special chars)
    text = replace_emoticons(text)

    # 2. Remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # 3. Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # 4. Convert hashtags to words (#happy -> happy)
    text = re.sub(r'#(\w+)', r'\1', text)

    # 5. Normalize elongated words (soooo -> soo)
    text = normalize_elongated_words(text)

    # 6. Replace slang and abbreviations
    text = replace_slang(text)

    # 7. Remove special characters but keep emotional punctuation
    # Keep: letters, numbers, spaces, and ! ? . , ' - "
    text = re.sub(r"[^a-zA-Z0-9\s!?.,;:'\"\-]", '', text)

    # 8. Normalize repeated punctuation (!!!! -> !!)
    text = re.sub(r'([!?.]){2,}', r'\1\1', text)

    # 9. Normalize whitespace
    text = re.sub(r'\s+', ' ', text)

    # 10. Strip and lowercase
    text = text.strip().lower()

    return text

print(f"Loaded {len(SLANG_MAP)} slang + {len(EMOTICON_MAP)} emoticon mappings")

Loaded 157 slang + 23 emoticon mappings


## 3. Data Loading

In [4]:
GOEMOTIONS_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

NUM_LABELS_GO = len(GOEMOTIONS_LABELS)
label_to_id_go = {l: i for i, l in enumerate(GOEMOTIONS_LABELS)}
id_to_label_go = {i: l for l, i in label_to_id_go.items()}
print(f"GoEmotions: {NUM_LABELS_GO} labels")

GoEmotions: 28 labels


In [5]:
from datasets import load_dataset

def process_example(example):
        text = example['text']
        labels = example['labels']

        multi_hot = [0] * NUM_LABELS_GO
        for label_idx in labels:
            if 0 <= label_idx < NUM_LABELS_GO:
                multi_hot[label_idx] = 1

        return { 'text': clean_text(text), 'labels': multi_hot }

def load_goemotions():
    dataset = load_dataset("google-research-datasets/go_emotions", "simplified")
    train_ds = dataset['train'].map(process_example)
    val_ds = dataset['validation'].map(process_example)
    test_ds = dataset['test'].map(process_example)

    return train_ds, val_ds, test_ds

print("Loading GoEmotions...")
train_ds, val_ds, test_ds = load_goemotions()
print(f"Train: {len(train_ds)}, Val: {len(val_ds)}, Test: {len(test_ds)}")

print("Example: ", train_ds[15] )


Loading GoEmotions...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Train: 43410, Val: 5426, Test: 5427
Example:  {'text': 'shit, i guess i accidentally bought a pay-per-view boxing match', 'labels': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'id': 'edivtm3'}


## 4. TF-IDF Gating Mechanism

In [6]:
def regex_analyzer(text):
    text = text.lower()
    return re.findall(r"(?u)\b\w+\b", text)

class TFIDFGating:
    """TF-IDF gating as described in the EA-RoBERTa paper."""

    def __init__(self, threshold=2.5, min_tokens=4):
        self.threshold = threshold
        self.min_tokens = min_tokens
        self.vectorizer = TfidfVectorizer( norm=None, analyzer=regex_analyzer )
        self._is_fitted = False

    def fit(self, texts):
        print(f"Fitting TF-IDF on {len(texts):,} samples...")

        self.vectorizer.fit(texts)
        self._is_fitted = True
        print(f"  Vocabulary: {len(self.vectorizer.vocabulary_):,} words")
        return self

    def get_word_scores(self, text):

        analyzer = self.vectorizer.build_analyzer()
        words = analyzer(text)
        if not words:
            return [], np.array([])

        tfidf = self.vectorizer.transform([text.lower()])
        vocab = self.vectorizer.vocabulary_

        scores = []
        for word in words:
            idx = vocab.get(word)
            scores.append(float(tfidf[0, idx]) if idx is not None else 0.0)

        return words, np.array(scores, dtype=np.float32)

    def compute_gates(self, text, normalize=True):

      # Compute TF-IDF for that text (using the fitted vectorizer)
      words, scores = self.get_word_scores(text)
      if len(words) == 0:
          return words, np.array([])

      keep = scores >= self.threshold
      if keep.sum() < self.min_tokens:
          if len(scores) >= self.min_tokens:
              top_k = np.argsort(-scores)[:self.min_tokens]
              keep = np.zeros_like(keep, dtype=bool)
              keep[top_k] = True
          else:
              keep = np.ones_like(keep, dtype=bool)

      gates = scores.copy()
      gates[~keep] = 0.0

      if normalize and gates.max() > 0:
          gates = gates / gates.max()

      return words, gates.astype(np.float32)

    def get_stats(self, texts):
          """Get filtering statistics."""

          orig_lens, filt_lens = [], []
          for text in texts:
              words, scores = self.get_word_scores(text)
              if len(words) == 0:
                  continue

              # Apply same logic as compute_gates
              keep = scores >= self.threshold
              if keep.sum() < self.min_tokens:
                  if len(scores) >= self.min_tokens:
                      top_k = np.argsort(-scores)[:self.min_tokens]
                      keep = np.zeros_like(keep, dtype=bool)
                      keep[top_k] = True
                  else:
                      keep = np.ones_like(keep, dtype=bool)

              orig_lens.append(len(words))
              filt_lens.append(keep.sum())

          return {
              'avg_original': np.mean(orig_lens),
              'avg_filtered': np.mean(filt_lens),
              'avg_retention': np.mean([f/o if o > 0 else 1 for o, f in zip(orig_lens, filt_lens)])
          }

# Fit TF-IDF on oversampled train data
tfidf_gating = TFIDFGating( threshold=CONFIG_GOEMOTIONS['tfidf_threshold'], min_tokens=CONFIG_GOEMOTIONS['tfidf_min_tokens'] )
tfidf_gating.fit([item['text'] for item in train_ds])

Fitting TF-IDF on 43,410 samples...
  Vocabulary: 26,662 words


## 5. Dataset & DataLoader

In [7]:
tokenizer = RobertaTokenizerFast.from_pretrained(CONFIG_GOEMOTIONS['model_name'])
print(f"Tokenizer: {CONFIG_GOEMOTIONS['model_name']}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Tokenizer: roberta-base


In [8]:
class MultiLabelEmotionDataset(Dataset):
    """Dataset for multi-label emotion classification."""

    def __init__(self, data, tokenizer, tfidf_gating=None, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.tfidf_gating = tfidf_gating
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        labels = item['labels']  # Multi-hot vector

        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding=False,
            return_offsets_mapping=True,
            return_tensors=None
        )

        result = {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'labels': labels  # Multi-hot [0,1,0,1,...]
        }

        if self.tfidf_gating is not None:
            words, word_gates = self.tfidf_gating.compute_gates(text)
            token_gates = self._map_gates_to_tokens(
                text, words, word_gates, encoding['offset_mapping']
            )
            result['gates'] = token_gates

        return result

    def _map_gates_to_tokens(self, text, words, word_gates, offsets):
        """Same as before."""
        seq_len = len(offsets)
        token_gates = [1.0] * seq_len

        if len(words) == 0:
            return token_gates

        word_spans = []
        pos = 0
        text_lower = text.lower()
        for word in words:
            start = text_lower.find(word, pos)
            if start != -1:
                word_spans.append((start, start + len(word)))
                pos = start + len(word)
            else:
                word_spans.append(None)

        for token_idx, (char_start, char_end) in enumerate(offsets):
            if char_start == char_end:
                token_gates[token_idx] = 1.0
                continue

            for word_idx, span in enumerate(word_spans):
                if span is None:
                    continue
                ws, we = span
                if not (char_end <= ws or char_start >= we):
                    token_gates[token_idx] = float(word_gates[word_idx])
                    break

        return token_gates

In [9]:
class MultiLabelDataCollator:
    """Collator for multi-label classification."""

    def __init__(self, tokenizer):
        self.pad_id = tokenizer.pad_token_id

    def __call__(self, features):
        max_len = max(len(f['input_ids']) for f in features)

        batch = { 'input_ids': [], 'attention_mask': [], 'labels': [] }
        has_gates = 'gates' in features[0]
        if has_gates:
            batch['gates'] = []

        for f in features:
            pad_len = max_len - len(f['input_ids'])
            batch['input_ids'].append(f['input_ids'] + [self.pad_id] * pad_len)
            batch['attention_mask'].append(f['attention_mask'] + [0] * pad_len)
            batch['labels'].append(f['labels'])
            if has_gates:
                batch['gates'].append(f['gates'] + [0.0] * pad_len)

        batch['input_ids'] = torch.tensor(batch['input_ids'], dtype=torch.long)
        batch['attention_mask'] = torch.tensor(batch['attention_mask'], dtype=torch.long)
        # IMPORTANT: Float for BCEWithLogitsLoss
        batch['labels'] = torch.tensor(batch['labels'], dtype=torch.float32)
        if has_gates:
            batch['gates'] = torch.tensor(batch['gates'], dtype=torch.float32)

        return batch

In [10]:
# Create datasets
gating = tfidf_gating if CONFIG_GOEMOTIONS['use_tfidf_gating'] else None

train_dataset_go = MultiLabelEmotionDataset(train_ds, tokenizer, gating, CONFIG_GOEMOTIONS['max_length'])
val_dataset_go = MultiLabelEmotionDataset(val_ds, tokenizer, gating, CONFIG_GOEMOTIONS['max_length'])
test_dataset_go = MultiLabelEmotionDataset(test_ds, tokenizer, gating, CONFIG_GOEMOTIONS['max_length'])
data_collator = MultiLabelDataCollator(tokenizer)

print(f"✅ Datasets created:")
print(f"   Train: {len(train_dataset_go):,}")
print(f"   Val:   {len(val_dataset_go):,}")
print(f"   Test:  {len(test_dataset_go):,}")
print(f"   TF-IDF gating: {CONFIG_GOEMOTIONS['use_tfidf_gating']}")

✅ Datasets created:
   Train: 43,410
   Val:   5,426
   Test:  5,427
   TF-IDF gating: True


## 6. EA-RoBERTa Model (ESA + TF-IDF Gating)

In [11]:
class ESALayer(nn.Module):
    """
    Emotion-Specific Attention Layer.

    From paper: "The ESA layer introduces emotion-aware weighting by learning
    an emotion-specific importance vector that modulates attention distribution."
    """

    def __init__(self, hidden_dim: int, num_heads: int = 12, max_len: int = 512):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.max_len = max_len

        # Learned positional encoding P ∈ R[max_len, H]
        self.pos_emb = nn.Embedding(max_len, hidden_dim)

        # Multi-head attention
        self.mha = nn.MultiheadAttention( embed_dim=hidden_dim, num_heads=num_heads, batch_first=True )

        # Learnable scaling vector S (emotion-specific importance)
        self.S = nn.Parameter(torch.ones(hidden_dim))

    def forward(self, E, attention_mask=None):
        """
        Args:
            E: [B, L, H] - RoBERTa embeddings
            attention_mask: [B, L] - 1=real, 0=pad
        Returns:
            Z_final: [B, L, H] - ESA-refined embeddings
        """
        B, L, H = E.shape

        # Add positional encoding
        positions = torch.arange(L, device=E.device).unsqueeze(0).expand(B, L)
        P = self.pos_emb(positions)
        E_input = E + P

        # Create key padding mask for MHA (True = ignore)
        key_padding_mask = None
        if attention_mask is not None:
            key_padding_mask = (attention_mask == 0)

        # Self-attention
        Z, attention_weights = self.mha(
            E_input, E_input, E_input,
            key_padding_mask=key_padding_mask,
            need_weights=True,
            average_attn_weights=False
        )

        # Emotion-specific scaling
        Z_scaled = Z * self.S

        # Re-add positional encoding
        Z_final = Z_scaled + P

        return Z_final, attention_weights

In [12]:
class EARoBERTaMultiLabel(nn.Module):
    """EA-RoBERTa for multi-label emotion classification."""

    def __init__(self, model_name, num_labels, dropout=0.3, use_gates=True, dense_dim=256):
        super().__init__()

        self.num_labels = num_labels
        self.use_gates = use_gates

        self.roberta = RobertaModel.from_pretrained(model_name)
        hidden_size = self.roberta.config.hidden_size
        cfg = self.roberta.config

        # ESA layer
        self.esa = ESALayer(
            hidden_dim=hidden_size,
            num_heads=cfg.num_attention_heads,
            max_len=cfg.max_position_embeddings
        )

        # Classification head
        self.dropout = nn.Dropout(dropout)
        self.dense = nn.Sequential(
            nn.Linear(2 * hidden_size, dense_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.classifier = nn.Linear(dense_dim, num_labels)

        # BCE loss for multi-label
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, input_ids=None, attention_mask=None, labels=None,
                inputs_embeds=None, gates=None, **kwargs):

        # Apply TF-IDF gating
        if self.use_gates and inputs_embeds is None and input_ids is not None and gates is not None:
            word_embeds = self.roberta.embeddings.word_embeddings(input_ids)
            gates = gates.unsqueeze(-1).type_as(word_embeds)                  # [B,L,1]
            inputs_embeds = word_embeds * gates
            input_ids = None

        outputs = self.roberta(
            input_ids=input_ids,
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            return_dict=True
        )

        E = outputs.last_hidden_state
        Z, _ = self.esa(E, attention_mask)

        # Pooling
        cls_output = Z[:, 0, :]
        mask = attention_mask.unsqueeze(-1).type_as(Z)
        mean_output = (Z * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)
        pooled = torch.cat([cls_output, mean_output], dim=-1)

        # Classification
        pooled = self.dropout(pooled)
        hidden = self.dense(pooled)
        logits = self.classifier(hidden)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

In [13]:
# Create model
model_go = EARoBERTaMultiLabel(
    model_name=CONFIG_GOEMOTIONS['model_name'],
    num_labels=NUM_LABELS_GO,
    dropout=CONFIG_GOEMOTIONS['dropout'],
    use_gates=CONFIG_GOEMOTIONS['use_tfidf_gating']
)
model_go.to(device)

total_params = sum(p.numel() for p in model_go.parameters())
trainable_params = sum(p.numel() for p in model_go.parameters() if p.requires_grad)

print(f"\n✅ EA-RoBERTa created on {device}")
print(f"   Total params: {total_params:,}")
print(f"   Trainable: {trainable_params:,}")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ EA-RoBERTa created on cuda
   Total params: 127,804,188
   Trainable: 127,804,188


In [14]:
# Test forward pass
model_go.train()
batch = next(iter(DataLoader(train_dataset_go, batch_size=4, collate_fn=data_collator)))
batch = {k: v.to(device) for k, v in batch.items()}

with torch.no_grad():
    out = model_go(**batch)

print(f"✅ Forward pass OK")
print(f"   Logits: {out.logits.shape}")
print(f"   Loss: {out.loss.item():.4f}")

✅ Forward pass OK
   Logits: torch.Size([4, 28])
   Loss: 0.6942


## 7. Training

In [15]:
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss

def compute_multilabel_metrics(pred):
    """Compute metrics for multi-label classification."""

    logits = pred.predictions
    labels = pred.label_ids

    # Apply sigmoid and threshold
    probs = 1 / (1 + np.exp(-logits))  # Sigmoid
    preds = (probs > CONFIG_GOEMOTIONS['multilabel_threshold']).astype(int)

    return {
        'f1_micro': f1_score(labels, preds, average='micro', zero_division=0),
        'f1_macro': f1_score(labels, preds, average='macro', zero_division=0),
        'f1_weighted': f1_score(labels, preds, average='weighted', zero_division=0),
        'precision_micro': precision_score(labels, preds, average='micro', zero_division=0),
        'recall_micro': recall_score(labels, preds, average='micro', zero_division=0),
        'hamming_loss': hamming_loss(labels, preds),
    }

In [16]:
training_args = TrainingArguments(
    output_dir='./results/ea_roberta_multiLabel',

    # Training
    num_train_epochs=CONFIG_GOEMOTIONS['num_epochs'],
    per_device_train_batch_size=CONFIG_GOEMOTIONS['batch_size'],
    per_device_eval_batch_size=CONFIG_GOEMOTIONS['batch_size'],
    learning_rate=CONFIG_GOEMOTIONS['learning_rate'],
    weight_decay=CONFIG_GOEMOTIONS['weight_decay'],
    warmup_ratio=CONFIG_GOEMOTIONS['warmup_ratio'],

    # Evaluation
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1_micro',
    greater_is_better=True,

    # Logging
    logging_dir='./logs/ea_roberta',
    logging_steps=100,

    # Optimization
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=4,
    remove_unused_columns=False,

    # Other
    report_to='none',
    seed=CONFIG_GOEMOTIONS['seed'],
    save_total_limit=2
)

print("✅ Training arguments configured")

✅ Training arguments configured


In [17]:
trainer = Trainer(
    model=model_go,
    args=training_args,
    train_dataset=train_dataset_go,
    eval_dataset=val_dataset_go,
    data_collator=data_collator,
    compute_metrics=compute_multilabel_metrics
)

print("✅ Trainer created")
print(f"   Train batches: {len(train_dataset_go) // CONFIG_GOEMOTIONS['batch_size']:,}")
print(f"   Total steps: {len(train_dataset_go) // CONFIG_GOEMOTIONS['batch_size'] * CONFIG_GOEMOTIONS['num_epochs']:,}")

✅ Trainer created
   Train batches: 1,356
   Total steps: 13,560


In [18]:
print("=" * 60)
print(" STARTING EA-RoBERTa TRAINING")
print("=" * 60)
print(f"   Device: {device}")
print(f"   Epochs: {CONFIG_GOEMOTIONS['num_epochs']}")
print(f"   Batch size: {CONFIG_GOEMOTIONS['batch_size']}")
print(f"   Learning rate: {CONFIG_GOEMOTIONS['learning_rate']}")
print(f"   TF-IDF Gating: {CONFIG_GOEMOTIONS['use_tfidf_gating']}")
print("=" * 60)

train_result = trainer.train()

print("\n✅ Training complete!")
print(f"   Loss: {train_result.training_loss:.4f}")
print(f"   Time: {train_result.metrics['train_runtime']:.1f}s")

 STARTING EA-RoBERTa TRAINING
   Device: cuda
   Epochs: 10
   Batch size: 32
   Learning rate: 2e-05
   TF-IDF Gating: True


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Weighted,Precision Micro,Recall Micro,Hamming Loss,Runtime,Samples Per Second,Steps Per Second
1,0.141600,0.117133,0.401935,0.099285,0.300314,0.712405,0.279937,0.034984,3.254400,1667.270000,52.237000
2,0.102600,0.093291,0.482861,0.227093,0.406219,0.747617,0.356583,0.032074,3.368300,1610.906000,50.471000
3,0.092900,0.086813,0.552757,0.347852,0.502747,0.728158,0.445455,0.030271,3.746100,1448.426000,45.380000
4,0.086700,0.087520,0.554212,0.396741,0.519688,0.705697,0.456270,0.030824,3.174900,1709.039000,53.545000
5,0.077200,0.086884,0.575079,0.446351,0.554058,0.681633,0.497335,0.030863,3.225700,1682.135000,52.702000
6,0.075000,0.087641,0.574520,0.443262,0.554691,0.676290,0.499373,0.031061,3.174800,1709.082000,53.547000
7,0.068800,0.092530,0.570526,0.454562,0.554796,0.648314,0.509404,0.032206,3.230000,1679.895000,52.632000
8,0.062900,0.093453,0.582514,0.453442,0.566497,0.647374,0.529467,0.031870,3.256400,1666.234000,52.204000
9,0.060600,0.094935,0.578114,0.456918,0.566182,0.640732,0.526646,0.032278,3.168500,1712.480000,53.653000
10,0.056900,0.096908,0.574470,0.459315,0.561681,0.635035,0.524451,0.032627,3.471100,1563.189000,48.976000



✅ Training complete!
   Loss: 0.0938
   Time: 727.6s


In [19]:
model_go.eval()
batch_dev = {k: v.to(device) for k, v in batch.items()}

with torch.no_grad():
    out_g = model_go(**batch_dev)

batch_ones = dict(batch_dev)
batch_ones["gates"] = torch.ones_like(batch_dev["gates"])

with torch.no_grad():
    out_1 = model_go(**batch_ones)

diff = (out_g.logits - out_1.logits).abs().mean().item()
print("Mean |logit diff| (gates vs ones):", diff)


Mean |logit diff| (gates vs ones): 0.5493392944335938


In [20]:
# Save model
trainer.save_model('./models/ea_roberta')
tokenizer.save_pretrained('./models/ea_roberta')

# Save config
with open('./models/ea_roberta/config.json', 'w') as f:
    json.dump(CONFIG_GOEMOTIONS, f, indent=2)

print("✅ Model saved to ./models/ea_roberta/")

✅ Model saved to ./models/ea_roberta/


## 8. Evaluation

In [21]:
print("\n" + "=" * 60)
print("📊 EVALUATION")
print("=" * 60)

# Validation
val_results = trainer.evaluate(val_dataset_go)
print("\nValidation:")
for k, v in val_results.items():
    if isinstance(v, float):
        print(f"   {k}: {v:.4f}")

# Test
test_results = trainer.evaluate(test_dataset_go)
print("\nTest:")
for k, v in test_results.items():
    if isinstance(v, float):
        print(f"   {k}: {v:.4f}")


📊 EVALUATION



Validation:
   eval_loss: 0.0935
   eval_f1_micro: 0.5825
   eval_f1_macro: 0.4534
   eval_f1_weighted: 0.5665
   eval_precision_micro: 0.6474
   eval_recall_micro: 0.5295
   eval_hamming_loss: 0.0319
   eval_runtime: 3.2306
   eval_samples_per_second: 1679.5530
   eval_steps_per_second: 52.6210
   epoch: 10.0000

Test:
   eval_loss: 0.0942
   eval_f1_micro: 0.5761
   eval_f1_macro: 0.4589
   eval_f1_weighted: 0.5605
   eval_precision_micro: 0.6377
   eval_recall_micro: 0.5254
   eval_hamming_loss: 0.0322
   eval_runtime: 3.5767
   eval_samples_per_second: 1517.3400
   eval_steps_per_second: 47.5300
   epoch: 10.0000


In [22]:
import numpy as np
from sklearn.metrics import classification_report

pred = trainer.predict(test_dataset_go)
logits = pred.predictions  # (N, 28)

# y_true: (N, 28) multi-hot
y_true = np.array([test_dataset_go[i]["labels"] for i in range(len(test_dataset_go))])

# sigmoid -> probs
probs = 1 / (1 + np.exp(-logits))

# choose a threshold (start with 0.5, often 0.3-0.4 works better for GoEmotions)
thr = 0.5
y_pred = (probs >= thr).astype(int)  # (N, 28) multi-hot

label_names = [id_to_label_go[i] for i in range(NUM_LABELS_GO)]

print(classification_report(y_true, y_pred, target_names=label_names, zero_division=0))


                precision    recall  f1-score   support

    admiration       0.66      0.72      0.69       504
     amusement       0.77      0.82      0.79       264
         anger       0.59      0.39      0.47       198
     annoyance       0.47      0.21      0.29       320
      approval       0.41      0.40      0.41       351
        caring       0.47      0.38      0.42       135
     confusion       0.45      0.38      0.41       153
     curiosity       0.53      0.38      0.44       284
        desire       0.67      0.37      0.48        83
disappointment       0.49      0.17      0.25       151
   disapproval       0.47      0.32      0.38       267
       disgust       0.66      0.38      0.48       123
 embarrassment       0.79      0.41      0.54        37
    excitement       0.53      0.38      0.44       103
          fear       0.69      0.68      0.68        78
     gratitude       0.93      0.90      0.92       352
         grief       0.00      0.00      0.00  

# LIVE TESTING INTERFACE

### Word Importance Explainability (LOO Method)

In the interactive tester, explainability is implemented using a **text perturbation method** called **Leave-One-Out (LOO)**. The model is treated as a *black box* — we don’t inspect its internal layers, we only observe how its predictions change when the input is modified.

#### Steps of the method:

1. **Clean the text** (remove special characters, normalize the format).
2. **Tokenize the text** using the RoBERTa tokenizer.
3. **Compute TF-IDF gates externally** to assign an importance score to each word, and map those scores to the corresponding subtokens.
4. **Run the model on the full text** → this gives us the reference prediction score, also called **baseline emotion probability**.
5. **Remove one word at a time from the text**, rebuild the sentence without it, and send it again to the model.
6. **Recompute emotion probabilities** for the modified text.
7. **Calculate the importance of the removed word** as:

   ```
   importance = baseline_emotion_probability − emotion_probability_without_word
   ```

#### Score interpretation:

* **importance > 0** → the emotion probability **decreased** when the word was removed
  → meaning the word was **supporting that emotion**.
* **importance < 0** → the emotion probability **increased** when the word was removed
  → meaning the word was **suppressing or opposing that emotion**.
* **importance ≈ 0** → very small or no change
  → meaning the word had **minimal emotional impact (near-neutral)**.

#### Visualization in the UI:

* 🔴 **Red** = positive contribution (emotion-supporting)
* 🔵 **Blue** = negative contribution (emotion-blocking/opposing)
* ⚪ **Gray** = minimal or neutral impact

---

### Why use this method?

* It is **easy to understand and implement**
* It shows the **real measurable influence of each word on emotion probabilities**
* It is **gradient-free**, so it works for any model as a reliable **black-box explanation**
* This approach is widely used in explainability research and scientific papers



In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import torch
import numpy as np

class LiveMultiLabelTester:
    """Interactive multi-label emotion tester with per-emotion word importance."""

    def __init__(self, model, tokenizer, tfidf_gating, id_to_label, device, config):
        self.model = model
        self.tokenizer = tokenizer
        self.tfidf_gating = tfidf_gating
        self.id_to_label = id_to_label
        self.label_to_id = {v: k for k, v in id_to_label.items()}
        self.device = device
        self.config = config
        self.threshold = config['multilabel_threshold']
        self.model.eval()

    # =========================================================================
    # PREDICTION
    # =========================================================================
    def predict(self, text):
        """Predict multiple emotions."""

        if not text.strip():
            return [], {}, ([], [])

        cleaned = clean_text(text)
        words, gates = self.tfidf_gating.compute_gates(cleaned)

        encoding = self.tokenizer( cleaned, truncation=True, max_length=self.config['max_length'], padding='max_length', return_tensors='pt' )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        gates_tensor = self._compute_token_gates(cleaned, encoding)

        with torch.no_grad():
            outputs = self.model( input_ids=input_ids, attention_mask=attention_mask, gates=gates_tensor )
            probs = torch.sigmoid(outputs.logits)[0]

        all_probs = {self.id_to_label[i]: probs[i].item() for i in range(len(probs))}
        predictions = [label for label, prob in all_probs.items() if prob > self.threshold]

        return predictions, all_probs, (words, gates)

    def _compute_token_gates(self, text, encoding):
        """Compute token-level gates from word-level TF-IDF."""

        words, word_gates = self.tfidf_gating.compute_gates(text)
        seq_len = encoding['input_ids'].shape[1]
        token_gates = [1.0] * seq_len

        if len(words) == 0:
            return torch.tensor([token_gates], dtype=torch.float32).to(self.device)

        enc_with_offsets = self.tokenizer( text, return_offsets_mapping=True, truncation=True, max_length=self.config['max_length'] )
        offsets = enc_with_offsets['offset_mapping']

        word_spans = []
        pos = 0
        text_lower = text.lower()
        for word in words:
            start = text_lower.find(word, pos)
            if start != -1:
                word_spans.append((start, start + len(word)))
                pos = start + len(word)
            else:
                word_spans.append(None)

        for token_idx, (char_start, char_end) in enumerate(offsets):
            if char_start == char_end:
                continue
            for word_idx, span in enumerate(word_spans):
                if span is None:
                    continue
                ws, we = span
                if not (char_end <= ws or char_start >= we):
                    token_gates[token_idx] = float(word_gates[word_idx])
                    break

        token_gates = token_gates + [0.0] * (self.config['max_length'] - len(token_gates))
        token_gates = token_gates[:self.config['max_length']]

        return torch.tensor([token_gates], dtype=torch.float32).to(self.device)

    # =========================================================================
    # WORD IMPORTANCE (Perturbation-based - most reliable)
    # =========================================================================
    def _get_emotion_prob(self, text, emotion):
        """Get probability for a specific emotion."""
        if not text.strip():
            return 0.0

        encoding = self.tokenizer(text, truncation=True, max_length=self.config['max_length'], padding='max_length', return_tensors='pt' )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        gates_tensor = self._compute_token_gates(text, encoding)

        with torch.no_grad():
            outputs = self.model( input_ids=input_ids, attention_mask=attention_mask, gates=gates_tensor )
            probs = torch.sigmoid(outputs.logits)[0]

        target_idx = self.label_to_id[emotion]
        return probs[target_idx].item()

    def _get_all_probs(self, text):
        """Get all emotion probabilities."""

        if not text.strip():
            return {self.id_to_label[i]: 0.0 for i in range(len(self.id_to_label))}

        encoding = self.tokenizer( text, truncation=True, max_length=self.config['max_length'], padding='max_length', return_tensors='pt' )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        gates_tensor = self._compute_token_gates(text, encoding)

        with torch.no_grad():
            outputs = self.model( input_ids=input_ids, attention_mask=attention_mask, gates=gates_tensor )
            probs = torch.sigmoid(outputs.logits)[0]

        return {self.id_to_label[i]: probs[i].item() for i in range(len(probs))}

    def compute_word_importance(self, text, target_emotion):
        """
        Compute how much each word contributes to a specific emotion.

        Method: Leave-One-Out (Perturbation)
        - Remove each word one at a time
        - Measure how much the emotion probability drops
        - Higher drop = more important word for that emotion
        """
        cleaned = clean_text(text)
        words = cleaned.split()

        if len(words) == 0:
            return { 'words': [], 'importance': np.array([]), 'emotion': target_emotion, 'probability': 0.0 }

        # Get baseline prediction with all words
        base_prob = self._get_emotion_prob(cleaned, target_emotion)

        importance_scores = []

        for i in range(len(words)):
            # Create text WITHOUT word i
            words_without_i = words[:i] + words[i+1:]
            modified_text = ' '.join(words_without_i)

            if modified_text.strip():
                new_prob = self._get_emotion_prob(modified_text, target_emotion)
            else:
                new_prob = 0.0

            # Importance = probability DROP when word is removed
            # Positive = word HELPS this emotion
            # Negative = word HURTS this emotion
            importance = base_prob - new_prob
            importance_scores.append(importance)

        importance_scores = np.array(importance_scores)

        return {'words': words, 'importance': importance_scores, 'emotion': target_emotion, 'probability': base_prob }

    def compute_all_emotions_importance(self, text):
        """
        Compute word importance for ALL emotions at once.
        More efficient than calling compute_word_importance for each emotion.
        """

        cleaned = clean_text(text)
        words = cleaned.split()

        if len(words) == 0:
            return {}

        # Get baseline probabilities for all emotions
        base_probs = self._get_all_probs(cleaned)

        # For each word removal, get all emotion probs
        all_importance = {emotion: [] for emotion in self.id_to_label.values()}

        for i in range(len(words)):
            words_without_i = words[:i] + words[i+1:]
            modified_text = ' '.join(words_without_i)

            if modified_text.strip():
                new_probs = self._get_all_probs(modified_text)
            else:
                new_probs = {e: 0.0 for e in self.id_to_label.values()}

            # Compute importance for each emotion
            for emotion in self.id_to_label.values():
                importance = base_probs[emotion] - new_probs[emotion]
                all_importance[emotion].append(importance)

        # Convert to numpy arrays
        results = {}
        for emotion in self.id_to_label.values():
            results[emotion] = {'words': words, 'importance': np.array(all_importance[emotion]), 'probability': base_probs[emotion] }

        return results

    # =========================================================================
    # VISUALIZATION
    # =========================================================================
    def _importance_to_color(self, score, max_abs_score):
        """Convert importance score to color."""
        if max_abs_score == 0:
            return "#040404"

        # Normalize to [-1, 1]
        norm_score = score / max_abs_score

        if norm_score > 0:
            # Positive = helps emotion = RED (intensity based on score)
            intensity = int(255 * (1 - abs(norm_score) * 0.7))
            return f'rgb(255, {intensity}, {intensity})'
        elif norm_score < 0:
            # Negative = hurts emotion = BLUE
            intensity = int(255 * (1 - abs(norm_score) * 0.7))
            return f'rgb({intensity}, {intensity}, 255)'
        else:
            return "#040404"

    def visualize_word_importance(self, text, emotion):
        """Create HTML visualization for one emotion."""
        result = self.compute_word_importance(text, emotion)

        words = result['words']
        scores = result['importance']
        prob = result['probability']

        if len(scores) == 0:
            return HTML(f'<p>No words to analyze for {emotion}</p>')

        max_abs = max(abs(scores.max()), abs(scores.min()), 0.001)

        html = f'''
        <div style="margin: 15px 0; padding: 15px; border: 1px solid #ddd; border-radius: 8px;">
            <h4 style="margin: 0 0 10px 0;">
                <b>{emotion.upper()}</b>
                <span style="color: #666; font-weight: normal;">({prob:.1%} probability)</span>
            </h4>
            <div style="line-height: 2.5;">
        '''

        for word, score in zip(words, scores):
            color = self._importance_to_color(score, max_abs)

            # Text color based on background
            text_color = '#000' if abs(score/max_abs) < 0.5 else '#fff'

            # Show + or - sign
            sign = '+' if score > 0 else ''

            html += f'''
            <span style="
                background: {color};
                color: {text_color};
                padding: 4px 10px;
                margin: 3px;
                border-radius: 5px;
                display: inline-block;
                font-size: 14px;
                cursor: help;
            " title="{sign}{score:.3f} ({sign}{score/prob*100:.1f}% relative impact)">
                {word}
            </span>
            '''

        html += '''
            </div>
            <div style="margin-top: 10px; font-size: 11px; color: #888;">
                <span style="background: rgb(255,100,100); color: white; padding: 2px 6px; border-radius: 3px;">Red = supports emotion</span>
                <span style="background: rgb(100,100,255); color: white; padding: 2px 6px; border-radius: 3px; margin-left: 5px;">Blue = opposes emotion</span>
                <span style="background: #eee; padding: 2px 6px; border-radius: 3px; margin-left: 5px;">Gray = neutral</span>
            </div>
        </div>
        '''

        return HTML(html)

    def visualize_all_emotions(self, text, top_k=5):
        """Visualize word importance for top K emotions."""
        cleaned = clean_text(text)
        predictions, all_probs, _ = self.predict(text)

        # Get top K emotions by probability
        sorted_emotions = sorted(all_probs.items(), key=lambda x: -x[1])[:top_k]

        html = f'''
        <div style="font-family: Arial, sans-serif;">
            <h2>Word Importance Analysis</h2>
            <p style="color: #666; margin-bottom: 20px;">
                Input: "<i>{text}</i>"<br>
                Cleaned: "<i>{cleaned}</i>"
            </p>
        '''

        # Compute importance for all emotions at once (efficient)
        all_results = self.compute_all_emotions_importance(text)

        for emotion, prob in sorted_emotions:
            result = all_results[emotion]
            words = result['words']
            scores = result['importance']

            if len(scores) == 0:
                continue

            max_abs = max(abs(scores.max()), abs(scores.min()), 0.001)

            # Check if above threshold
            status = "✅" if prob > self.threshold else "○"

            html += f'''
            <div style="margin: 15px 0; padding: 12px; border-left: 4px solid {"#4CAF50" if prob > self.threshold else "#ddd"}; background: #fafafa;">
                <h4 style="margin: 0 0 8px 0;">
                    {status} <b>{emotion}</b>
                    <span style="color: #666; font-weight: normal;">({prob:.1%})</span>
                </h4>
                <div style="line-height: 2.3;">
            '''

            for word, score in zip(words, scores):
                color = self._importance_to_color(score, max_abs)
                text_color = '#000' if abs(score/max_abs) < 0.5 else '#fff'

                html += f'''
                <span style="
                    background: {color};
                    color: {text_color};
                    padding: 3px 8px;
                    margin: 2px;
                    border-radius: 4px;
                    display: inline-block;
                    font-size: 13px;
                " title="impact: {score:+.3f}">{word}</span>
                '''

            html += '</div></div>'

        # Legend
        html += '''
        <div style="margin-top: 20px; padding: 10px; background: #f5f5f5; border-radius: 5px; font-size: 12px;">
            <b>Legend:</b><br>
            🔴 <b>Red</b> = word INCREASES this emotion's probability (supporting)<br>
            🔵 <b>Blue</b> = word DECREASES this emotion's probability (opposing)<br>
            ⚪ <b>Gray</b> = word has minimal impact<br><br>
            <i>Hover over words to see exact impact values</i>
        </div>
        </div>
        '''

        return HTML(html)

    def show_comparison_table(self, text, emotions=None):
        """Show a table comparing word importance across emotions."""
        cleaned = clean_text(text)
        words = cleaned.split()

        if len(words) == 0:
            return HTML('<p>No words to analyze</p>')

        all_results = self.compute_all_emotions_importance(text)

        # Select emotions to show
        if emotions is None:
            # Top 5 by probability
            _, all_probs, _ = self.predict(text)
            emotions = [e for e, _ in sorted(all_probs.items(), key=lambda x: -x[1])[:5]]

        html = '''
        <style>
            .importance-table { border-collapse: collapse; font-size: 12px; }
            .importance-table th, .importance-table td {
                padding: 8px;
                border: 1px solid #ddd;
                text-align: center;
            }
            .importance-table th { background: #f0f0f0; }
            .importance-table td:first-child { font-weight: bold; text-align: left; }
        </style>
        <h3> Word Importance Comparison</h3>
        <table class="importance-table">
            <tr>
                <th>Word</th>
        '''

        for emotion in emotions:
            prob = all_results[emotion]['probability']
            html += f'<th>{emotion}<br><small>({prob:.0%})</small></th>'

        html += '</tr>'

        for i, word in enumerate(words):
            html += f'<tr><td>{word}</td>'

            for emotion in emotions:
                score = all_results[emotion]['importance'][i]
                max_abs = max(abs(all_results[emotion]['importance'].max()),
                             abs(all_results[emotion]['importance'].min()), 0.001)

                color = self._importance_to_color(score, max_abs)

                html += f'<td style="background: {color};">{score:+.2f}</td>'

            html += '</tr>'

        html += '</table>'

        return HTML(html)

    # =========================================================================
    # INTERACTIVE WIDGET
    # =========================================================================
    def run_interactive(self):
        """Launch interactive widget interface."""

        # Text input
        text_input = widgets.Textarea(
            value='',
            placeholder='Enter text to analyze emotions...',
            layout=widgets.Layout(width='100%', height='100px')
        )

        # Controls
        threshold_slider = widgets.FloatSlider(
            value=self.threshold,
            min=0.1,
            max=0.9,
            step=0.05,
            description='Threshold:',
            layout=widgets.Layout(width='300px')
        )

        top_k_slider = widgets.IntSlider(
            value=5,
            min=1,
            max=15,
            step=1,
            description='Top K:',
            layout=widgets.Layout(width='200px')
        )

        emotion_dropdown = widgets.Dropdown(
            options=sorted(self.id_to_label.values()),
            value='joy',
            description='Emotion:',
            layout=widgets.Layout(width='200px')
        )

        # Buttons
        predict_btn = widgets.Button(
            description='Predict',
            button_style='primary',
            layout=widgets.Layout(width='110px')
        )

        explain_all_btn = widgets.Button(
            description='Explain All',
            button_style='success',
            layout=widgets.Layout(width='120px')
        )

        explain_one_btn = widgets.Button(
            description='Explain One',
            button_style='info',
            layout=widgets.Layout(width='120px')
        )

        compare_btn = widgets.Button(
            description='Compare',
            button_style='warning',
            layout=widgets.Layout(width='110px')
        )

        output = widgets.Output()

        # =====================================================================
        # Button handlers
        # =====================================================================
        def on_predict(b):
            self.threshold = threshold_slider.value
            with output:
                clear_output()
                text = text_input.value

                if not text.strip():
                    display(HTML('<p style="color: red;">Please enter some text!</p>'))
                    return

                predictions, all_probs, (words, gates) = self.predict(text)

                # Header
                if predictions:
                    pred_html = ', '.join([f'<b style="color: #4CAF50;">{p}</b>' for p in predictions])
                    display(HTML(f'<h2>Detected: {pred_html}</h2>'))
                else:
                    display(HTML('<h2 style="color: #999;">No emotions above threshold</h2>'))

                # Top emotions bar chart
                sorted_probs = sorted(all_probs.items(), key=lambda x: -x[1])[:10]

                html = '<div style="font-family: monospace;">'
                for emotion, prob in sorted_probs:
                    bar_width = int(prob * 250)
                    color = '#4CAF50' if prob > self.threshold else '#ccc'
                    check = '✓' if prob > self.threshold else ''

                    html += f'''
                    <div style="margin: 4px 0; display: flex; align-items: center;">
                        <span style="width: 120px;">{emotion}</span>
                        <div style="background: {color}; width: {bar_width}px; height: 18px; border-radius: 3px;"></div>
                        <span style="margin-left: 10px; width: 60px;">{prob:.1%}</span>
                        <span style="color: green;">{check}</span>
                    </div>
                    '''
                html += '</div>'
                display(HTML(html))

        def on_explain_all(b):
            self.threshold = threshold_slider.value
            with output:
                clear_output()
                text = text_input.value

                if not text.strip():
                    display(HTML('<p style="color: red;">Please enter some text!</p>'))
                    return

                display(HTML('<p>⏳ Computing word importance for all emotions...</p>'))
                clear_output(wait=True)

                display(self.visualize_all_emotions(text, top_k=top_k_slider.value))

        def on_explain_one(b):
            with output:
                clear_output()
                text = text_input.value
                emotion = emotion_dropdown.value

                if not text.strip():
                    display(HTML('<p style="color: red;">Please enter some text!</p>'))
                    return

                display(HTML(f'<p>Computing word importance for <b>{emotion}</b>...</p>'))
                clear_output(wait=True)

                display(self.visualize_word_importance(text, emotion))

        def on_compare(b):
            self.threshold = threshold_slider.value
            with output:
                clear_output()
                text = text_input.value

                if not text.strip():
                    display(HTML('<p style="color: red;">Please enter some text!</p>'))
                    return

                display(HTML('<p>Building comparison table...</p>'))
                clear_output(wait=True)

                # Get top K emotions
                _, all_probs, _ = self.predict(text)
                top_emotions = [e for e, _ in sorted(all_probs.items(), key=lambda x: -x[1])[:top_k_slider.value]]

                display(self.show_comparison_table(text, emotions=top_emotions))

        # Connect handlers
        predict_btn.on_click(on_predict)
        explain_all_btn.on_click(on_explain_all)
        explain_one_btn.on_click(on_explain_one)
        compare_btn.on_click(on_compare)

        # Layout
        display(HTML('''
        <h1>🎭 EA-RoBERTa Multi-Label Emotion Analyzer</h1>
        <p style="color: #666;">Analyze emotions and see which words contribute to each prediction</p>
        '''))

        display(text_input)
        display(widgets.HBox([predict_btn, explain_all_btn, explain_one_btn, compare_btn]))
        display(widgets.HBox([emotion_dropdown, top_k_slider, threshold_slider]))
        display(output)


# =============================================================================
# INITIALIZE AND RUN
# =============================================================================
tester = LiveMultiLabelTester(
    model=model_go,
    tokenizer=tokenizer,
    tfidf_gating=tfidf_gating,
    id_to_label=id_to_label_go,
    device=device,
    config=CONFIG_GOEMOTIONS
)

tester.run_interactive()

ModuleNotFoundError: No module named 'ipywidgets'

* Yikes....Clueless on what happend here
* Wtf why would you do that tho?
* I regret saying that, it was embarrassing and wrong
* I love how you always care about me, you're the best.
